In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [4]:
# Wide and Deep Neural Network 
# With Keras Functional API

input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation='relu')(input_)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

model.compile(loss='mse', optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
mse_test

Epoch 1/30
363/363 [==============================] - 3s 7ms/step - loss: 4.0079 - val_loss: 0.8114
Epoch 2/30
363/363 [==============================] - 1s 2ms/step - loss: 0.7780 - val_loss: 0.6802
Epoch 3/30
363/363 [==============================] - 1s 2ms/step - loss: 0.6623 - val_loss: 0.6205
Epoch 4/30
363/363 [==============================] - 1s 2ms/step - loss: 0.6138 - val_loss: 0.5758
Epoch 5/30
363/363 [==============================] - 1s 3ms/step - loss: 0.5922 - val_loss: 0.5442
Epoch 6/30
363/363 [==============================] - 1s 3ms/step - loss: 0.5463 - val_loss: 0.5179
Epoch 7/30
363/363 [==============================] - 1s 3ms/step - loss: 0.5238 - val_loss: 0.4966
Epoch 8/30
363/363 [==============================] - 1s 3ms/step - loss: 0.5115 - val_loss: 0.4794
Epoch 9/30
363/363 [==============================] - 1s 2ms/step - loss: 0.4849 - val_loss: 0.4655
Epoch 10/30
363/363 [==============================] - 1s 2ms/step - loss: 0.4866 - val_loss: 0.4565

0.3819921314716339

In [7]:
# Wide and Deep With Multiple Inputs

input_A = keras.layers.Input(shape=[5], name='wide_input')
input_B = keras.layers.Input(shape=[6], name='deep_input')
hidden1 = keras.layers.Dense(30, activation='relu')(input_B)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name='output')(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

model.compile(loss='mse', optimizer=keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20, validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
mse_test

Epoch 1/20
363/363 [==============================] - 2s 4ms/step - loss: 3.0133 - val_loss: 1.0014
Epoch 2/20
363/363 [==============================] - 1s 4ms/step - loss: 0.8990 - val_loss: 0.6927
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.6752 - val_loss: 0.5919
Epoch 4/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5796 - val_loss: 0.5352
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5272 - val_loss: 0.4956
Epoch 6/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4892 - val_loss: 0.4690
Epoch 7/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4700 - val_loss: 0.4514
Epoch 8/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4584 - val_loss: 0.4390
Epoch 9/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4576 - val_loss: 0.4284
Epoch 10/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4375 - val_loss: 0.4211

0.5632278323173523

In [8]:
y_pred = model.predict((X_new_A, X_new_B))
y_pred

array([[1.8431767],
       [3.0132978],
       [1.5972428]], dtype=float32)

In [9]:
# Adding a Aux Output

input_A = keras.layers.Input(shape=[5], name='wide_input')
input_B = keras.layers.Input(shape=[6], name='deep_input')
hidden1 = keras.layers.Dense(30, activation='relu')(input_B)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name='output')(concat)
aux_output = keras.layers.Dense(1, name='aux_output')(hidden2)
model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

model.compile(loss=['mse', 'mse'], loss_weights=[0.9, 0.1], optimizer='sgd')

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20, validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))
mse_test = model.evaluate([X_test_A, X_test_B], [y_test, y_test])
mse_test

Epoch 1/20
363/363 [==============================] - 2s 3ms/step - loss: 1.6520 - output_loss: 1.4748 - aux_output_loss: 3.2470 - val_loss: 0.5270 - val_output_loss: 0.4530 - val_aux_output_loss: 1.1924
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5240 - output_loss: 0.4634 - aux_output_loss: 1.0694 - val_loss: 0.4559 - val_output_loss: 0.4124 - val_aux_output_loss: 0.8479
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4786 - output_loss: 0.4360 - aux_output_loss: 0.8623 - val_loss: 0.4295 - val_output_loss: 0.4010 - val_aux_output_loss: 0.6861
Epoch 4/20
363/363 [==============================] - 1s 4ms/step - loss: 0.4242 - output_loss: 0.3940 - aux_output_loss: 0.6958 - val_loss: 0.4146 - val_output_loss: 0.3924 - val_aux_output_loss: 0.6144
Epoch 5/20
363/363 [==============================] - 2s 4ms/step - loss: 0.4411 - output_loss: 0.4170 - aux_output_loss: 0.6580 - val_loss: 0.4206 - val_output_loss: 0.4025 - val_aux_

[0.46235427260398865, 0.39619624614715576, 1.057776689529419]

In [11]:
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])
y_pred_main

array([[2.195557 ],
       [3.239079 ],
       [1.5656145]], dtype=float32)